In [7]:
import requests
import pandas as pd
from requests.exceptions import HTTPError, ConnectionError, SSLError, Timeout, TooManyRedirects
from time import sleep
from random import choice, randint

class DEGIROScraper:

    def __init__(self, url_pattern, sleep=True, content_parser=None):
        
        self.url_pattern = url_pattern #stocks, bonds, funds, or etfs
        
        self.sleep = sleep
        self.content_parser = content_parser
    
    def scrape_urls(self, url): #Funcion que se aprovecha de la API interna de GIRO para extrar información de sus productos financieros
        
        total = 1
        product_list = []

        #Parametros de lo URL
        params = {'requireTotal':'True', #Le solicitamos que nos devuelva un atributo con el total de resultado  
        'offset': 0 ,                    #Punto de donde empieza la busqueda
        'limit':'1000',                  #Resultados totales por get (1000 es el maximo)
        'intAccount':'55010998',         #Valor asociado a mi cuenta de DEGIRO (es siempre el mismo)
        'sessionId': '693E145AFA74299FE0E1C8DD65ABE1FB.prod_b_128_4'} #Valor asociado a cada sesion que se inicia en DEGIRO
                                                                        #Cambia con cada login
        
        headers = {'user-agent' : self.get_random_useragent(),  #Funcion para obtener un user-agent al azar
                  'refer': self.get_referer(),                  #Funcion para obtener un referer segun el producto financiero
                  'accept': 'application/json, text/plain, */*', #Rellenamos estos requests headers que suele pedir DEGIRO,
                  'accept-encoding':'gzip, deflate, br',           #gracias a esto es mas facil disimular nuestra procedencia.
                  'accept-language':'en-US,en;q=0.9,ca;q=0.8,es;q=0.7,pt;q=0.6'}                  
        
        while params['offset'] < total: #Se usa el offset como el indice del while total se define mas adelante.
            
            try: #Creamos un try,except por si se produce algun error con el get
                
                response = requests.get(url, headers=headers, params=params) #Peticion get con parametros y headers definidos

                if response.status_code != 200: #Si el codigo no es 200 que se cree una exception
                    print(response.status_code)
                    raise Exception('Status Code Not Succesful')

                result = response.json() #Convertimos la respuesta en un diccionario

            except (HTTPError, ConnectionError, SSLError, Timeout, TooManyRedirects) as err: #Atrapamos estos errores si se producen
                print(err)

            except Exception as err:
                print(err)
            finally:
                              
                total = result['total']  #Total siempre sera igual al maximo de resultados 
                product_list.extend(result['products']) #Extendemos la lista de los resusltados obtenidos con los de la siguiente iteracion 
                params['offset'] += 1000 #Sumamos 1000 a nuestro indice

                self.sleep_interval() #Intervalo de espera al azar.


        self.output_results(product_list) #Exportamos el Dataset obtenido

    def sleep_interval(self):   #Funcion que generar tiempos de esperar al azar entre GETs
        if self.sleep == True:
            delays = [7, 4, 6, 2, 10, 19]    
            sleep(choice(delays))
        else:
            pass
    
    def get_random_useragent(self):  #Funcion que se encarga de generar user-agents al azar a partir de un txt
        with open('user-agent.txt') as f:
            lines = f.readlines()
            
        return lines[randint(0,len(lines)-1)].strip()
    
    def get_referer(self): #Selecciona el referer segun el tipo de scrapeo que se este haciendo
        referer = {'stocks':'https://trader.degiro.nl/beta-trader/#/products?productType=1',    
                'bonds':'https://trader.degiro.nl/beta-trader/#/products?productType=2',
                'funds':'https://trader.degiro.nl/beta-trader/#/products?productType=13',
                'etfs':'https://trader.degiro.nl/beta-trader/#/products?productType=131'}
    
        return referer[self.url_pattern]
    
    
    def output_results(self, product_list): #Funcion que exporta el resultado obtenido a un archivo .csv
        
        pd.DataFrame(product_list).to_csv( self.url_pattern + '.csv', sep=',', index=False)    


    def kickstart(self): #Funcion inicilizadora del proceso de scrapeo

        url = 'https://trader.degiro.nl/product_search/secure/v5/%s?'  #URL base de DEGIRO para la busqueda de productos
        self.scrape_urls(url%self.url_pattern)                         #En %s se introduce el producto a buscar: bonds,funds,stocks or etfs

        

In [8]:
Bonds = DEGIROScraper(url_pattern='bonds', sleep=True)
Bonds.kickstart()

In [9]:
Funds = DEGIROScraper(url_pattern='funds', sleep=True)
Funds.kickstart()

In [11]:
ETFs = DEGIROScraper(url_pattern='etfs', sleep=True)
ETFs.kickstart()

In [12]:
Stocks = DEGIROScraper(url_pattern='stocks', sleep=True)
Stocks.kickstart()